In [3]:
# install requirments
!pip install -r ./requirements.txt
!pip install --upgrade jupyter ipywidgets

  Using cached editdistance-0.6.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (282 kB)
  Using cached matplotlib-3.6.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached sentencepiece-0.1.97-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached wandb-0.13.7-py2.py3-none-any.whl (1.9 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached kiwisolver-1.4.4-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
  Using cached contourpy-1.0.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (296 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached GitPython-3.1.30-py3-none-any.whl (184 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached 

In [1]:
# install fairseq
!git submodule add https://github.com/pytorch/fairseq.git
!git submodule init
!cd fairseq && git checkout 9a1c497
!pip install --upgrade ./fairseq/

Cloning into '/home/stceum/simple_NMT_on_transformer/code/fairseq'...
remote: Enumerating objects: 34294, done.
remote: Total 34294 (delta 0), reused 0 (delta 0), pack-reused 34294
Receiving objects: 100% (34294/34294), 23.76 MiB | 1.52 MiB/s, done.
Resolving deltas: 100% (24966/24966), done.
Note: switching to '9a1c497'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 9a1c4970 Make Hydra logging work with DDP (#1568)
Processing ./fairseq
  Installing build dependencies ... done
  Getting requirements to build 

In [4]:
import sys
import pdb
import pprint
import logging
import os
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import numpy as np
import tqdm.auto as tqdm
from pathlib import Path
from argparse import Namespace
from fairseq import utils

import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.
